## Columnas importantes para el modelo predictivo de inventario

Este notebook se enfoca en generar datos sintéticos únicamente para las columnas más relevantes que utiliza el modelo de predicción de inventario, incluyendo:

- Variables de stock: StockActual, StockMinimo, StockRecomendado, StockSeguridad
- Variables de demanda: DemandaPromedio, Importacion, Exportacion
- Variables temporales: TiempoReposicion
- Variables económicas: PrecioVenta, PrecioProveedor
- Variables categóricas: Categoría, Temporada

La generación de datos se hace de forma realista considerando relaciones lógicas entre estas variables.

# Generador de Datos Sintéticos para Modelo de Inventario

Este notebook genera datos sintéticos realistas utilizando la biblioteca Faker para alimentar un modelo de predicción de inventario. Se enfoca exclusivamente en las columnas importantes que necesita el modelo para su entrenamiento.

In [ ]:
# Instalación de bibliotecas necesarias
!pip install faker pandas numpy matplotlib seaborn

# Importar bibliotecas
import pandas as pd
import numpy as np
from faker import Faker
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random
import os

# Configurar Faker para datos en español
fake = Faker(['es_MX'])
Faker.seed(42)  # Para reproducibilidad
np.random.seed(42)
random.seed(42)

print("Librerías importadas correctamente")

## 4. Generación de datos para la tabla de Artículos

Generamos datos de artículos con diferentes categorías y precios.

# Unir datos de artículos e inventario para crear el dataset del modelo
df_modelo = pd.merge(inventario_df, articulos_df[['Articulo_ID', 'Categoria', 'Temporada', 'PrecioProveedor', 'PrecioVenta']], on='Articulo_ID')

# Calcular algunas métricas adicionales importantes para el modelo
df_modelo['DiasInventario'] = df_modelo['StockActual'] / df_modelo['DemandaPromedio']
df_modelo['RotacionInventario'] = df_modelo['Exportacion'] / df_modelo['StockActual']
df_modelo['MargenGanancia'] = (df_modelo['PrecioVenta'] - df_modelo['PrecioProveedor']) / df_modelo['PrecioProveedor']

# Mostrar estadísticas descriptivas de las variables importantes
print("Estadísticas descriptivas de variables críticas para el modelo:")
df_modelo[['StockActual', 'StockMinimo', 'StockRecomendado', 'DiasInventario', 
           'DemandaPromedio', 'TiempoReposicion', 'RotacionInventario', 'MargenGanancia']].describe()

In [ ]:
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from faker import Faker

fake = Faker()

# Generar datos para la tabla de Artículos
def generar_articulos(num_articulos=300):
    categorias = [
        "Electrónica", "Ropa", "Hogar", "Deportes", "Juguetes", "Libros", "Alimentos", 
        "Bebidas", "Cosméticos", "Ferretería", "Automotriz", "Jardinería", "Medicina"
    ]
    
    temporadas = ["Primavera", "Verano", "Otoño", "Invierno", "Todo el año"]
    
    articulos = []
    
    for i in range(1, num_articulos + 1):
        categoria = random.choice(categorias)
        
        # Generar nombre según la categoría
        if categoria == "Electrónica":
            prefijos = ["Smartphone", "Tablet", "Laptop", "Monitor", "Teclado", "Mouse", "Auriculares"]
            marcas = ["Samsung", "Apple", "Xiaomi", "Huawei", "LG", "Sony", "HP", "Dell"]
            nombre = f"{random.choice(prefijos)} {random.choice(marcas)} {fake.word().capitalize()}"
        elif categoria == "Ropa":
            tipos = ["Camiseta", "Pantalón", "Vestido", "Falda", "Chaqueta", "Suéter", "Zapatos"]
            marcas = ["Zara", "H&M", "Nike", "Adidas", "Levi's", "Gap", "Forever 21"]
            nombre = f"{random.choice(tipos)} {random.choice(marcas)} {fake.word().capitalize()}"
        else:
            nombre = f"{fake.word().capitalize()} {fake.word().capitalize()}"
        
        # Precios adecuados según categoría
        if categoria in ["Electrónica", "Automotriz"]:
            precio_proveedor = round(random.uniform(1000, 15000), 2)
        elif categoria in ["Ropa", "Deportes", "Cosméticos"]:
            precio_proveedor = round(random.uniform(100, 2000), 2)
        else:
            precio_proveedor = round(random.uniform(20, 500), 2)
        
        # Margen de ganancia entre 20% y 60%
        margen = random.uniform(1.2, 1.6)
        precio_venta = round(precio_proveedor * margen, 2)
        
        # Asignar temporada con mayor probabilidad a "Todo el año"
        temporada = random.choices(temporadas, weights=[0.15, 0.15, 0.15, 0.15, 0.4])[0]
        
        articulo = {
            "Articulo_ID": i,
            "Nombre": nombre,
            "Categoria": categoria,
            "PrecioProveedor": precio_proveedor,
            "PrecioVenta": precio_venta,
            "Temporada": temporada
        }
        articulos.append(articulo)
    
    return pd.DataFrame(articulos)

articulos_df = generar_articulos(300)
print(f"Generados {len(articulos_df)} artículos")
articulos_df.head()

# Visualización de la correlación entre variables importantes para el modelo
plt.figure(figsize=(12, 10))

# Seleccionar solo columnas numéricas importantes
columnas_importantes = [
    'StockActual', 'StockMinimo', 'StockRecomendado', 'StockSeguridad', 
    'DemandaPromedio', 'TiempoReposicion', 'Importacion', 'Exportacion',
    'DiasInventario', 'RotacionInventario', 'MargenGanancia'
]

# Calcular correlaciones
corr_matrix = df_modelo[columnas_importantes].corr()

# Crear un heatmap
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)

plt.title('Matriz de Correlación de Variables Importantes para el Modelo')
plt.tight_layout()
plt.show()

In [ ]:
# Visualización de la distribución de artículos por categoría
plt.figure(figsize=(12, 8))
sns.countplot(y='Categoria', data=articulos_df)
plt.title('Distribución de Artículos por Categoría')
plt.xlabel('Cantidad')
plt.ylabel('Categoría')
plt.tight_layout()
plt.show()

# Visualización de la distribución de precios por categoría
plt.figure(figsize=(14, 8))
sns.boxplot(x='Categoria', y='PrecioVenta', data=articulos_df)
plt.title('Distribución de Precios de Venta por Categoría')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Categoría')
plt.ylabel('Precio de Venta (MXN)')
plt.tight_layout()
plt.show()

## 4. Codificación de Variables Categóricas

Realizamos la codificación de variables categóricas para que puedan ser utilizadas por el modelo.

## 5. Generación de datos para la tabla de Inventario

Generamos datos de inventario que incluyan información sobre stock, importaciones, exportaciones y otros indicadores importantes para el modelo predictivo.

# Codificar variables categóricas para el modelo
def preparar_datos_finales():
    # Convertir variables categóricas a numéricas
    df_final = pd.get_dummies(df_modelo, columns=['Categoria', 'Temporada'], drop_first=True)
    
    # Seleccionar solo las columnas importantes para el modelo
    columnas_modelo = [
        # Variables de stock
        'StockActual', 'StockMinimo', 'StockRecomendado', 'StockSeguridad',
        # Variables de demanda
        'DemandaPromedio', 'Importacion', 'Exportacion', 
        # Variables derivadas
        'DiasInventario', 'RotacionInventario',
        # Variables económicas
        'PrecioVenta', 'PrecioProveedor', 'MargenGanancia',
        # Variables temporales
        'TiempoReposicion',
        # Variables categóricas codificadas
        'Categoria_Automotriz', 'Categoria_Bebidas', 'Categoria_Cosmética',
        'Categoria_Deportes', 'Categoria_Electrónica', 'Categoria_Hogar', 
        'Categoria_Ropa', 'Temporada_Invierno', 'Temporada_Otoño', 
        'Temporada_Primavera', 'Temporada_Todo el año', 'Temporada_Verano'
    ]
    
    # Seleccionar solo las columnas que existen en el dataframe
    columnas_existentes = [col for col in columnas_modelo if col in df_final.columns]
    df_final = df_final[columnas_existentes]
    
    return df_final

# Crear dataset final para el modelo
dataset_modelo = preparar_datos_finales()
print(f"Dataset para el modelo creado con {dataset_modelo.shape[0]} filas y {dataset_modelo.shape[1]} columnas")
dataset_modelo.head()

In [ ]:
# Generar datos para la tabla de Inventario
def generar_inventario():
    # Vamos a crear registros de inventario para cada artículo en múltiples ubicaciones
    inventarios = []
    inventario_id = 1
    
    # Filtrar solo ubicaciones que son almacenes o centros de distribución
    ubicaciones_almacen = ubicaciones_df[ubicaciones_df['Tipo'].isin(['Almacén', 'Centro de Distribución'])]
    
    for articulo_id in articulos_df['Articulo_ID']:
        # No todos los artículos están en todas las ubicaciones
        num_ubicaciones = random.randint(1, min(5, len(ubicaciones_almacen)))
        ubicaciones_seleccionadas = random.sample(ubicaciones_almacen['Location_ID'].tolist(), num_ubicaciones)
        
        # Obtener categoría y temporada del artículo para ajustar los parámetros de inventario
        articulo = articulos_df[articulos_df['Articulo_ID'] == articulo_id].iloc[0]
        categoria = articulo['Categoria']
        temporada = articulo['Temporada']
        
        for location_id in ubicaciones_seleccionadas:
            # Ajustar el stock según la categoría y la temporada
            if categoria in ["Electrónica", "Automotriz"]:
                stock_base = random.randint(5, 50)  # Artículos caros, menos stock
            elif categoria in ["Alimentos", "Bebidas"]:
                stock_base = random.randint(50, 300)  # Artículos de consumo frecuente
            else:
                stock_base = random.randint(20, 150)  # Stock intermedio
            
            # Ajustar según temporada
            if temporada != "Todo el año":
                # Verificar si estamos en temporada (simulando)
                temporada_actual = random.choice(["Primavera", "Verano", "Otoño", "Invierno", "Todo el año"])
                if temporada == temporada_actual:
                    stock_base = int(stock_base * random.uniform(1.5, 2.5))  # Más stock en temporada
                else:
                    stock_base = int(stock_base * random.uniform(0.3, 0.7))  # Menos stock fuera de temporada
            
            # Stock mínimo y recomendado
            stock_minimo = max(3, int(stock_base * random.uniform(0.1, 0.2)))
            stock_recomendado = int(stock_base * random.uniform(1.2, 1.5))
            
            # Importación y exportación (movimientos)
            importacion = int(stock_base * random.uniform(0.05, 0.3))
            exportacion = int(stock_base * random.uniform(0.05, 0.25))
            
            # Fechas de última importación y exportación
            fecha_ultima_importacion = fake.date_between(start_date='-3m', end_date='today')
            fecha_ultima_exportacion = fake.date_between(start_date='-3m', end_date='today')
            
            # Margen de ganancia
            margen_ganancia = round(random.uniform(0.15, 0.60), 2)  # Entre 15% y 60%
            
            # Tiempo de reposición en días
            if categoria in ["Electrónica", "Automotriz"]:
                tiempo_reposicion = round(random.uniform(7, 30), 1)  # Más tiempo para importar
            else:
                tiempo_reposicion = round(random.uniform(1, 14), 1)
            
            # Stock de seguridad
            stock_seguridad = max(2, int(stock_minimo * random.uniform(1.1, 1.5)))
            
            # Demanda promedio diaria
            if categoria in ["Alimentos", "Bebidas"]:
                demanda_promedio = round(random.uniform(5, 25), 1)  # Alta rotación
            elif categoria in ["Electrónica", "Automotriz"]:
                demanda_promedio = round(random.uniform(0.5, 5), 1)  # Baja rotación
            else:
                demanda_promedio = round(random.uniform(1, 10), 1)  # Rotación media
            
            inventario = {
                "Inventario_ID": inventario_id,
                "Articulo_ID": articulo_id,
                "Location_ID": location_id,
                "StockActual": stock_base,
                "Importacion": importacion,
                "Exportacion": exportacion,
                "StockMinimo": stock_minimo,
                "StockRecomendado": stock_recomendado,
                "FechaUltimaImportacion": fecha_ultima_importacion,
                "FechaUltimaExportacion": fecha_ultima_exportacion,
                "MargenGanancia": margen_ganancia,
                "TiempoReposicion": tiempo_reposicion,
                "StockSeguridad": stock_seguridad,
                "DemandaPromedio": demanda_promedio
            }
            
            inventarios.append(inventario)
            inventario_id += 1
    
    return pd.DataFrame(inventarios)

inventario_df = generar_inventario()
print(f"Generados {len(inventario_df)} registros de inventario")
inventario_df.head()

## 5. Exportar los Datos para el Modelo

Exportamos el dataset final con las variables importantes para que pueda ser utilizado en el entrenamiento del modelo.

In [ ]:
# Visualización de distribución de stock actual
plt.figure(figsize=(12, 6))
sns.histplot(inventario_df['StockActual'], bins=30, kde=True)
plt.title('Distribución de Stock Actual')
plt.xlabel('Stock Actual')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.show()

# Análisis de relación entre stock actual y stock mínimo
plt.figure(figsize=(10, 8))
sns.scatterplot(x='StockMinimo', y='StockActual', data=inventario_df)
plt.title('Relación entre Stock Mínimo y Stock Actual')
plt.xlabel('Stock Mínimo')
plt.ylabel('Stock Actual')
plt.tight_layout()
plt.show()

# Crear directorio para guardar los datos si no existe
import os
directorio_datos = '../data'
os.makedirs(directorio_datos, exist_ok=True)

# Guardar solo el dataset final para el modelo
dataset_modelo.to_csv(f'{directorio_datos}/dataset_modelo.csv', index=False)
print("Dataset para el modelo exportado correctamente a:", f'{directorio_datos}/dataset_modelo.csv')

## 6. Conclusiones

Hemos generado un dataset sintético enfocado exclusivamente en las variables importantes para el modelo de predicción de inventario:

1. **Variables de stock**: StockActual, StockMinimo, StockRecomendado, StockSeguridad
2. **Variables de demanda**: DemandaPromedio, Importacion, Exportacion
3. **Variables temporales**: TiempoReposicion
4. **Variables económicas**: PrecioVenta, PrecioProveedor, MargenGanancia
5. **Variables derivadas**: DiasInventario, RotacionInventario
6. **Variables categóricas codificadas**: Categoría, Temporada

Estas variables son las que tienen mayor impacto en la predicción del comportamiento del inventario y permiten al modelo realizar recomendaciones precisas sobre los niveles óptimos de stock.

In [ ]:
# Unir datos de artículos e inventario para un análisis más profundo
df_analisis = pd.merge(inventario_df, articulos_df, on='Articulo_ID')

# Calcular algunas métricas adicionales
df_analisis['DiasInventario'] = df_analisis['StockActual'] / df_analisis['DemandaPromedio']
df_analisis['ValorInventario'] = df_analisis['StockActual'] * df_analisis['PrecioVenta']
df_analisis['StockSobreMinimo'] = df_analisis['StockActual'] / df_analisis['StockMinimo']

# Mostrar estadísticas descriptivas
print("Estadísticas descriptivas de variables críticas:")
df_analisis[['StockActual', 'StockMinimo', 'StockRecomendado', 'DiasInventario', 
            'DemandaPromedio', 'TiempoReposicion', 'ValorInventario']].describe()

In [ ]:
# Visualización de días de inventario por categoría
plt.figure(figsize=(14, 8))
sns.boxplot(x='Categoria', y='DiasInventario', data=df_analisis)
plt.title('Días de Inventario por Categoría')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Categoría')
plt.ylabel('Días de Inventario')
plt.tight_layout()
plt.show()

# Visualización de la relación entre demanda promedio y stock actual
plt.figure(figsize=(12, 8))
sns.scatterplot(x='DemandaPromedio', y='StockActual', hue='Categoria', 
               size='PrecioVenta', sizes=(20, 200), alpha=0.7, data=df_analisis)
plt.title('Relación entre Demanda Promedio y Stock Actual por Categoría')
plt.xlabel('Demanda Promedio')
plt.ylabel('Stock Actual')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 7. Preparación de Datos para el Modelo Predictivo

Creamos un conjunto de datos estructurado para alimentar el modelo predictivo de inventario.

In [ ]:
# Crear un dataframe con las variables relevantes para el modelo
def preparar_datos_modelo():
    # Seleccionar columnas relevantes
    columnas_modelo = [
        'Articulo_ID', 'Location_ID', 'StockActual', 'StockMinimo', 'StockRecomendado',
        'Importacion', 'Exportacion', 'TiempoReposicion', 'StockSeguridad',
        'DemandaPromedio', 'PrecioProveedor', 'PrecioVenta', 'Categoria', 'Temporada'
    ]
    
    # Unir dataframes
    df_modelo = pd.merge(inventario_df, articulos_df, on='Articulo_ID')
    df_modelo = df_modelo[columnas_modelo]
    
    # Convertir variables categóricas a numéricas
    df_modelo = pd.get_dummies(df_modelo, columns=['Categoria', 'Temporada'], drop_first=True)
    
    # Calcular algunas características adicionales
    df_modelo['RotacionInventario'] = df_modelo['Exportacion'] / df_modelo['StockActual']
    df_modelo['MargenAbsoluto'] = df_modelo['PrecioVenta'] - df_modelo['PrecioProveedor']
    df_modelo['CoberturaDias'] = df_modelo['StockActual'] / df_modelo['DemandaPromedio']
    
    # Llenar valores NaN (por ejemplo, si la demanda promedio es 0)
    df_modelo = df_modelo.fillna(0)
    
    return df_modelo

# Crear el conjunto de datos para el modelo
df_modelo = preparar_datos_modelo()
print(f"Dataset para el modelo creado con {df_modelo.shape[0]} filas y {df_modelo.shape[1]} columnas")
df_modelo.head()

## 8. Exportar los Datos Generados

Guardamos los datos generados en archivos CSV para su uso posterior.

In [ ]:
# Crear directorio para guardar los datos si no existe
import os
directorio_datos = '../data'
os.makedirs(directorio_datos, exist_ok=True)

# Guardar cada conjunto de datos en un archivo CSV
roles_df.to_csv(f'{directorio_datos}/roles.csv', index=False)
ubicaciones_df.to_csv(f'{directorio_datos}/ubicaciones.csv', index=False)
usuarios_df.to_csv(f'{directorio_datos}/usuarios.csv', index=False)
articulos_df.to_csv(f'{directorio_datos}/articulos.csv', index=False)
inventario_df.to_csv(f'{directorio_datos}/inventario.csv', index=False)
df_modelo.to_csv(f'{directorio_datos}/dataset_modelo.csv', index=False)

print("Datos exportados correctamente a:", directorio_datos)

## 9. Conclusiones y Siguientes Pasos

1. **Datos Generados**: Hemos generado datos sintéticos realistas para roles, ubicaciones, usuarios, artículos e inventario.

2. **Características Críticas**: Identificamos y analizamos las características más relevantes para el modelo predictivo de inventario.

3. **Dataset Preparado**: Creamos un conjunto de datos estructurado que puede alimentar diversos modelos predictivos.

4. **Siguientes Pasos**:
   - Entrenar un modelo de predicción de stock recomendado
   - Implementar un sistema de alertas de inventario bajo
   - Desarrollar un panel de visualización para monitoreo en tiempo real

In [6]:
# Verificar la existencia y estructura de las tablas utilizadas
# Obtener lista de tablas en el esquema DBADMIN
def get_table_info(conn, table_name, schema='DBADMIN', check_case_variations=True):
    try:
        # Verificar si la tabla existe exactamente como se especificó
        table_exists_query = f"SELECT COUNT(*) as TABLE_EXISTS FROM SYS.TABLES WHERE SCHEMA_NAME = '{schema}' AND TABLE_NAME = '{table_name}'"
        table_exists_result = conn.sql(table_exists_query).collect()
        table_exists = table_exists_result['TABLE_EXISTS'][0] > 0
        
        if table_exists:
            # Obtener la estructura de la tabla
            columns_query = f"SELECT COLUMN_NAME, DATA_TYPE_NAME FROM SYS.TABLE_COLUMNS WHERE SCHEMA_NAME = '{schema}' AND TABLE_NAME = '{table_name}' ORDER BY POSITION"
            columns = conn.sql(columns_query).collect()
            
            return {
                'exists': True,
                'actual_name': table_name,
                'columns': columns
            }
        elif check_case_variations:
            # Si la tabla no existe con el nombre exacto, buscar variaciones de mayúsculas/minúsculas
            case_insensitive_query = f"SELECT TABLE_NAME FROM SYS.TABLES WHERE SCHEMA_NAME = '{schema}' AND UPPER(TABLE_NAME) = UPPER('{table_name}')"
            case_result = conn.sql(case_insensitive_query).collect()
            
            if not case_result.empty:
                actual_name = case_result['TABLE_NAME'][0]
                print(f"\n🔄 Tabla encontrada con variación de mayúsculas/minúsculas: '{table_name}' → '{actual_name}'")
                
                # Obtener la estructura de la tabla
                columns_query = f"SELECT COLUMN_NAME, DATA_TYPE_NAME FROM SYS.TABLE_COLUMNS WHERE SCHEMA_NAME = '{schema}' AND TABLE_NAME = '{actual_name}' ORDER BY POSITION"
                columns = conn.sql(columns_query).collect()
                
                return {
                    'exists': True,
                    'actual_name': actual_name,
                    'columns': columns,
                    'case_mismatch': True
                }
            
            # También buscar nombres similares que podrían ser lo que el usuario busca
            similar_query = f"""SELECT TABLE_NAME 
                            FROM SYS.TABLES 
                            WHERE SCHEMA_NAME = '{schema}' 
                            AND (
                                TABLE_NAME LIKE '%{table_name}%' OR
                                '{table_name}' LIKE '%' || TABLE_NAME || '%'
                            )
                            ORDER BY LENGTH(TABLE_NAME)"""
            similar_result = conn.sql(similar_query).collect()
            
            if not similar_result.empty:
                similar_tables = similar_result['TABLE_NAME'].tolist()
                print(f"\n🔍 No se encontró '{table_name}', pero hay tablas con nombres similares: {', '.join(similar_tables[:5])}")
                if len(similar_tables) > 5:
                    print(f"   ...y {len(similar_tables) - 5} más")
        
        return {'exists': False, 'columns': []}
    except Exception as e:
        print(f"Error al verificar tabla {table_name}: {e}")
        return {'exists': False, 'error': str(e), 'columns': []}

# Tablas a verificar - Solo se usarán tablas con sufijo 2
tablas = [
    'INVENTARIO2', 
    'HISTORIALPRODUCTOS2', 
    'ARTICULO2', 
    'ORDENESPRODUCTOS2', 
    'ORDENES2'
]

resultados = {}
for tabla in tablas:
    info = get_table_info(conn, tabla)
    resultados[tabla] = info
    
    if info['exists']:
        nombre_real = info.get('actual_name', tabla)
        if info.get('case_mismatch', False):
            print(f"✅ Tabla {tabla} existe como '{nombre_real}' (considere usar este nombre exacto)")
        else:
            print(f"✅ Tabla {tabla} existe")
        print(f"   Columnas: {len(info['columns'])}")
    else:
        print(f"❌ Tabla {tabla} no existe")
        
# Mostrar detalles de las columnas para las tablas que existen
print("\nDetalle de columnas por tabla:")
for tabla, info in resultados.items():
    if info['exists']:
        print(f"\n=== {tabla} ===")
        display(info['columns'])

# Verificar si las columnas mencionadas en el código existen en las tablas
tablas_columnas = {
    'INVENTARIO2': [
        'Inventario_ID', 'Articulo_ID', 'Location_ID', 'StockActual', 'StockMinimo', 
        'StockRecomendado', 'MargenGanancia', 'TiempoReposicion', 'StockSeguridad', 'DemandaPromedio'
    ],
    'HISTORIALPRODUCTOS2': ['Inventario_ID', 'Location_ID', 'Anio', 'Mes', 'Importacion', 'Exportacion', 'StockEnd', 'StockStart'],
    'ARTICULO2': ['Articulo_ID', 'Nombre', 'Categoria', 'PrecioProveedor', 'PrecioVenta', 'Temporada'],
    'ORDENESPRODUCTOS2': ['OrdenesProductos_ID', 'Inventario_ID', 'Orden_ID', 'Cantidad', 'PrecioUnitario'],
    'ORDENES2': ['Orden_ID', 'FechaCreacion', 'FechaEntrega', 'TipoOrden']
}

# Verificar que las columnas necesarias existan
print("\nVerificación de columnas necesarias:")
for tabla, columnas in tablas_columnas.items():
    if tabla in resultados and resultados[tabla]['exists']:
        columns_df = resultados[tabla]['columns']
        columnas_existentes = columns_df['COLUMN_NAME'].tolist()
        
        print(f"\n=== {tabla} ===")
        for col in columnas:
            if col in columnas_existentes:
                print(f"✅ {col}")
            else:
                # Buscar variaciones de mayúsculas/minúsculas para la columna
                variaciones = [c for c in columnas_existentes if c.upper() == col.upper()]
                if variaciones:
                    print(f"⚠️ {col} - Existe como '{variaciones[0]}'")
                else:
                    print(f"❌ {col} - No existe")
    else:
        print(f"\n=== {tabla} ===")
        print("❌ Tabla no existe")

✅ Tabla INVENTARIO2 existe
   Columnas: 14
✅ Tabla HISTORIALPRODUCTOS2 existe
   Columnas: 9
✅ Tabla HISTORIALPRODUCTOS2 existe
   Columnas: 9
✅ Tabla ARTICULO2 existe
   Columnas: 6
✅ Tabla ARTICULO2 existe
   Columnas: 6
✅ Tabla ORDENESPRODUCTOS2 existe
   Columnas: 5
✅ Tabla ORDENESPRODUCTOS2 existe
   Columnas: 5
✅ Tabla ORDENES2 existe
   Columnas: 17

Detalle de columnas por tabla:

=== INVENTARIO2 ===
✅ Tabla ORDENES2 existe
   Columnas: 17

Detalle de columnas por tabla:

=== INVENTARIO2 ===


,COLUMN_NAME,DATA_TYPE_NAME
0,INVENTARIO_ID,INTEGER
1,ARTICULO_ID,INTEGER
2,LOCATION_ID,INTEGER
3,STOCKACTUAL,INTEGER
4,IMPORTACION,INTEGER
5,EXPORTACION,INTEGER
6,STOCKMINIMO,INTEGER
7,STOCKRECOMENDADO,INTEGER
8,FECHAULTIMAIMPORTACION,DATE
9,FECHAULTIMAEXPORTACION,DATE



=== HISTORIALPRODUCTOS2 ===


,COLUMN_NAME,DATA_TYPE_NAME
0,HISTORIAL_ID,INTEGER
1,INVENTARIO_ID,INTEGER
2,LOCATION_ID,INTEGER
3,ANIO,INTEGER
4,MES,INTEGER
5,IMPORTACION,INTEGER
6,EXPORTACION,INTEGER
7,STOCKSTART,INTEGER
8,STOCKEND,INTEGER



=== ARTICULO2 ===


...


=== ORDENESPRODUCTOS2 ===


,COLUMN_NAME,DATA_TYPE_NAME
0,ORDENESPRODUCTOS_ID,INTEGER
1,ORDEN_ID,INTEGER
2,INVENTARIO_ID,INTEGER
3,CANTIDAD,INTEGER
4,PRECIOUNITARIO,DECIMAL



=== ORDENES2 ===


,COLUMN_NAME,DATA_TYPE_NAME
0,ORDEN_ID,INTEGER
1,CREADO_POR_ID,INTEGER
2,MODIFICADO_POR_ID,INTEGER
3,TIPOORDEN,NVARCHAR
4,ORGANIZACION,NVARCHAR
5,FECHACREACION,DATE
6,FECHAACEPTACION,DATE
7,FECHALIMITEPAGO,DATE
8,FECHAESTIMADAENTREGA,DATE
9,FECHAENTREGA,DATE



Verificación de columnas necesarias:

=== INVENTARIO2 ===
⚠️ Inventario_ID - Existe como 'INVENTARIO_ID'
⚠️ Articulo_ID - Existe como 'ARTICULO_ID'
⚠️ Location_ID - Existe como 'LOCATION_ID'
⚠️ StockActual - Existe como 'STOCKACTUAL'
⚠️ StockMinimo - Existe como 'STOCKMINIMO'
⚠️ StockRecomendado - Existe como 'STOCKRECOMENDADO'
⚠️ MargenGanancia - Existe como 'MARGENGANANCIA'
⚠️ TiempoReposicion - Existe como 'TIEMPOREPOSICION'
⚠️ StockSeguridad - Existe como 'STOCKSEGURIDAD'
⚠️ DemandaPromedio - Existe como 'DEMANDAPROMEDIO'

=== HISTORIALPRODUCTOS2 ===
⚠️ Inventario_ID - Existe como 'INVENTARIO_ID'
⚠️ Location_ID - Existe como 'LOCATION_ID'
⚠️ Anio - Existe como 'ANIO'
⚠️ Mes - Existe como 'MES'
⚠️ Importacion - Existe como 'IMPORTACION'
⚠️ Exportacion - Existe como 'EXPORTACION'
⚠️ StockEnd - Existe como 'STOCKEND'
⚠️ StockStart - Existe como 'STOCKSTART'

=== ARTICULO2 ===
⚠️ Articulo_ID - Existe como 'ARTICULO_ID'
⚠️ Nombre - Existe como 'NOMBRE'
⚠️ Categoria - Existe como 'CATEG

In [7]:
# API de DataFrame de HANA ML
# Cargar las tablas como DataFrames de HANA ML usando los nombres determinados en la celda anterior

# Obtener los nombres reales de las tablas (con mayúsculas/minúsculas correctas)
tabla_inventario = resultados['INVENTARIO2']['actual_name'] if resultados['INVENTARIO2']['exists'] else 'INVENTARIO2'
tabla_historial = resultados['HISTORIALPRODUCTOS2']['actual_name'] if resultados['HISTORIALPRODUCTOS2']['exists'] else 'HISTORIALPRODUCTOS2'
tabla_articulo = resultados['ARTICULO2']['actual_name'] if resultados['ARTICULO2']['exists'] else 'ARTICULO2'
tabla_ordenes_productos = resultados['ORDENESPRODUCTOS2']['actual_name'] if resultados['ORDENESPRODUCTOS2']['exists'] else 'ORDENESPRODUCTOS2'
tabla_ordenes = resultados['ORDENES2']['actual_name'] if resultados['ORDENES2']['exists'] else 'ORDENES2'

print(f"Cargando tablas con los nombres reales encontrados:")
print(f"- Inventario: {tabla_inventario}")
print(f"- Historial: {tabla_historial}")
print(f"- Articulo: {tabla_articulo}")
print(f"- Ordenes Productos: {tabla_ordenes_productos}")
print(f"- Ordenes: {tabla_ordenes}")

# Verificar que todas las tablas existan antes de continuar
tablas_requeridas = {
    'INVENTARIO2': tabla_inventario,
    'HISTORIALPRODUCTOS2': tabla_historial,
    'ARTICULO2': tabla_articulo, 
    'ORDENESPRODUCTOS2': tabla_ordenes_productos,
    'ORDENES2': tabla_ordenes
}

# Cargar las tablas como DataFrames de HANA ML
inventario_df = conn.table(tabla_inventario, schema='DBADMIN')
historial_df = conn.table(tabla_historial, schema='DBADMIN')
articulo_df = conn.table(tabla_articulo, schema='DBADMIN')
ordenes_productos_df = conn.table(tabla_ordenes_productos, schema='DBADMIN')
ordenes_df = conn.table(tabla_ordenes, schema='DBADMIN')

Cargando tablas con los nombres reales encontrados:
- Inventario: INVENTARIO2
- Historial: HISTORIALPRODUCTOS2
- Articulo: ARTICULO2
- Ordenes Productos: ORDENESPRODUCTOS2
- Ordenes: ORDENES2


In [8]:
# Transformar los DataFrames de HANA ML a DataFrames de pandas para visualización
inventario_df = inventario_df.collect()
historial_df = historial_df.collect()
articulo_df = articulo_df.collect()
ordenes_productos_df = ordenes_productos_df.collect()
ordenes_df = ordenes_df.collect()

In [9]:
inventario_df.head(5)

,INVENTARIO_ID,ARTICULO_ID,LOCATION_ID,STOCKACTUAL,IMPORTACION,EXPORTACION,STOCKMINIMO,STOCKRECOMENDADO,FECHAULTIMAIMPORTACION,FECHAULTIMAEXPORTACION,MARGENGANANCIA,TIEMPOREPOSICION,STOCKSEGURIDAD,DEMANDAPROMEDIO
0,1,1,8,120,0,0,30,50,2025-04-01,2025-04-20,50,5,20,40
1,2,2,8,100,0,0,25,50,2025-04-01,2025-04-19,50,5,20,35
2,3,3,8,80,0,0,20,50,2025-04-01,2025-04-18,50,5,15,30
3,4,4,9,60,0,0,15,50,2025-04-03,2025-04-21,50,6,10,25
4,5,5,9,70,0,0,20,50,2025-04-03,2025-04-21,50,6,12,28


In [10]:
historial_df.head(5)

,HISTORIAL_ID,INVENTARIO_ID,LOCATION_ID,ANIO,MES,IMPORTACION,EXPORTACION,STOCKSTART,STOCKEND
0,1,1,8,2025,4,120,80,150,120
1,2,2,8,2025,4,100,65,130,100


In [11]:
articulo_df.head(5)

,ARTICULO_ID,NOMBRE,CATEGORIA,PRECIOPROVEEDOR,PRECIOVENTA,TEMPORADA
0,1,Playera Básica S,Playera,50,100,Verano
1,2,Playera Básica M,Playera,50,100,Verano
2,3,Playera Básica G,Playera,50,100,Verano
3,4,Playera Estampada S,Playera,60,120,Verano
4,5,Playera Estampada M,Playera,60,120,Verano


In [12]:
ordenes_productos_df.head(5)

,ORDENESPRODUCTOS_ID,ORDEN_ID,INVENTARIO_ID,CANTIDAD,PRECIOUNITARIO
0,1,1,1,2,100
1,2,1,2,1,100
2,3,2,4,2,120
3,4,2,5,1,120
4,5,3,7,2,140


In [13]:
ordenes_df.head(5)

,ORDEN_ID,CREADO_POR_ID,MODIFICADO_POR_ID,TIPOORDEN,ORGANIZACION,FECHACREACION,FECHAACEPTACION,FECHALIMITEPAGO,FECHAESTIMADAENTREGA,FECHAENTREGA,ENTREGAATIEMPO,ESTADO,TOTAL,METODOPAGO_ID,DESCUENTOAPLICADO,TIEMPOREPOSICION,TIEMPOENTREGA
0,1,1,1.0,Venta,RopaExpress,2025-05-28,2025-05-31,2025-06-07,2025-06-12,2025-06-13,True,Completado,1500,1,100,2,1
1,2,2,2.0,Venta,RopaExpress,2025-05-28,2025-05-30,2025-06-04,2025-06-07,2025-06-08,True,Pendiente,750,2,50,3,1
2,3,3,3.0,Venta,RopaExpress,2025-05-28,2025-06-03,2025-06-09,2025-06-14,2025-06-15,False,En Reparto,500,3,0,5,2
3,4,1,1.0,Venta,RopaExpress,2025-05-28,2025-06-01,2025-06-05,2025-06-09,2025-06-10,True,Completado,2000,1,200,1,1
4,5,2,2.0,Venta,RopaExpress,2025-05-28,2025-06-02,2025-06-06,2025-06-11,2025-06-12,False,Pendiente,1200,2,100,4,2


In [14]:
# Merge de DataFrames de HANA ML
# Realizar el merge de los DataFrames de HANA ML
# First, check column name format and case
inventario_id_col = [col for col in inventario_df.columns if col.upper() == 'INVENTARIO_ID'][0]
articulo_id_col = [col for col in articulo_df.columns if col.upper() == 'ARTICULO_ID'][0]
orden_id_col = [col for col in ordenes_df.columns if col.upper() == 'ORDEN_ID'][0]

# Now use the exact column names for merging
df = inventario_df.merge(historial_df, on=inventario_id_col, how='inner') \
                  .merge(articulo_df, on=articulo_id_col, how='inner') \
                  .merge(ordenes_productos_df, on=inventario_id_col, how='inner') \
                  .merge(ordenes_df, on=orden_id_col, how='inner')

# Mostrar las primeras filas del DataFrame resultante
print("\nPrimeras filas del DataFrame resultante:")
df.head(5)


Primeras filas del DataFrame resultante:


,INVENTARIO_ID,ARTICULO_ID,LOCATION_ID_x,STOCKACTUAL,IMPORTACION_x,EXPORTACION_x,STOCKMINIMO,STOCKRECOMENDADO,FECHAULTIMAIMPORTACION,FECHAULTIMAEXPORTACION,...,FECHALIMITEPAGO,FECHAESTIMADAENTREGA,FECHAENTREGA,ENTREGAATIEMPO,ESTADO,TOTAL,METODOPAGO_ID,DESCUENTOAPLICADO,TIEMPOREPOSICION_y,TIEMPOENTREGA
0,1,1,8,120,0,0,30,50,2025-04-01,2025-04-20,...,2025-06-07,2025-06-12,2025-06-13,True,Completado,1500,1,100,2,1
1,1,1,8,120,0,0,30,50,2025-04-01,2025-04-20,...,2025-06-04,2025-06-08,2025-06-09,False,En Reparto,650,2,30,5,2
2,2,2,8,100,0,0,25,50,2025-04-01,2025-04-19,...,2025-06-07,2025-06-12,2025-06-13,True,Completado,1500,1,100,2,1
3,2,2,8,100,0,0,25,50,2025-04-01,2025-04-19,...,2025-06-06,2025-06-10,2025-06-11,True,Pendiente,2100,1,0,3,1


In [16]:
# Since we already have pandas DataFrames, we'll work with them directly
# First, let's clean up the column names to handle any duplicates from the merge

# Check if we have the merged DataFrame from cell 13
if 'df' in globals() and len(df) > 0:
    # The df from cell 13 already contains the merged data
    # Let's select and rename the columns we need for analysis
    
    # Create a clean DataFrame with the columns we need
    df_clean = df[[
        'INVENTARIO_ID',
        'ARTICULO_ID', 
        'LOCATION_ID_x',  # Using _x version from inventario table
        'STOCKACTUAL',
        'STOCKMINIMO',
        'STOCKRECOMENDADO',
        'MARGENGANANCIA',
        'TIEMPOREPOSICION_x',
        'STOCKSEGURIDAD',
        'DEMANDAPROMEDIO',
        'ANIO',
        'MES', 
        'IMPORTACION_y',  # Using _y version from historial table
        'EXPORTACION_y',
        'STOCKSTART',
        'STOCKEND',
        'CATEGORIA',
        'PRECIOPROVEEDOR',
        'PRECIOVENTA',
        'TEMPORADA',
        'CANTIDAD',
        'PRECIOUNITARIO',
        'FECHACREACION',
        'FECHAENTREGA',
        'TIPOORDEN'
    ]].copy()
    
    # Rename columns to remove suffixes and have consistent naming
    df_clean = df_clean.rename(columns={
        'LOCATION_ID_x': 'LOCATION_ID',
        'IMPORTACION_y': 'IMPORTACION',
        'EXPORTACION_y': 'EXPORTACION',
        'TIEMPOREPOSICION_x': 'TIEMPOREPOSICION'
    })
    
    # Calculate delivery time in days
    df_clean['FECHACREACION'] = pd.to_datetime(df_clean['FECHACREACION'])
    df_clean['FECHAENTREGA'] = pd.to_datetime(df_clean['FECHAENTREGA'])
    df_clean['TIEMPOENTREGADIAS'] = (df_clean['FECHAENTREGA'] - df_clean['FECHACREACION']).dt.days
    
    # Replace the original df with the cleaned version
    df = df_clean
    
    print(f"Datos procesados correctamente. Dimensiones: {df.shape}")
    print("\nPrimeras 5 filas:")
    display(df.head())
    
    print(f"\nColumnas disponibles: {list(df.columns)}")
else:
    print("No hay datos disponibles para procesar")

Datos procesados correctamente. Dimensiones: (4, 26)

Primeras 5 filas:


,INVENTARIO_ID,ARTICULO_ID,LOCATION_ID,STOCKACTUAL,STOCKMINIMO,STOCKRECOMENDADO,MARGENGANANCIA,TIEMPOREPOSICION,STOCKSEGURIDAD,DEMANDAPROMEDIO,...,CATEGORIA,PRECIOPROVEEDOR,PRECIOVENTA,TEMPORADA,CANTIDAD,PRECIOUNITARIO,FECHACREACION,FECHAENTREGA,TIPOORDEN,TIEMPOENTREGADIAS
0,1,1,8,120,30,50,50,5,20,40,...,Playera,50,100,Verano,2,100,2025-05-28,2025-06-13,Venta,16
1,1,1,8,120,30,50,50,5,20,40,...,Playera,50,100,Verano,1,100,2025-05-28,2025-06-09,Venta,12
2,2,2,8,100,25,50,50,5,20,35,...,Playera,50,100,Verano,1,100,2025-05-28,2025-06-13,Venta,16
3,2,2,8,100,25,50,50,5,20,35,...,Playera,50,100,Verano,2,100,2025-05-28,2025-06-11,Venta,14



Columnas disponibles: ['INVENTARIO_ID', 'ARTICULO_ID', 'LOCATION_ID', 'STOCKACTUAL', 'STOCKMINIMO', 'STOCKRECOMENDADO', 'MARGENGANANCIA', 'TIEMPOREPOSICION', 'STOCKSEGURIDAD', 'DEMANDAPROMEDIO', 'ANIO', 'MES', 'IMPORTACION', 'EXPORTACION', 'STOCKSTART', 'STOCKEND', 'CATEGORIA', 'PRECIOPROVEEDOR', 'PRECIOVENTA', 'TEMPORADA', 'CANTIDAD', 'PRECIOUNITARIO', 'FECHACREACION', 'FECHAENTREGA', 'TIPOORDEN', 'TIEMPOENTREGADIAS']


In [18]:
# Preparar datos de ventas mensuales para modelado predictivo

# Primero preparamos el DataFrame existente para extracción de datos de ventas
# Asegurémonos que los datos de HANA ya estén en un DataFrame de pandas
if 'df' not in globals() or df is None:
    print("Error: No hay datos cargados desde HANA")
else:
    # Crear un DataFrame de ventas usando datos reales de la base de datos
    df_ventas = df.copy()
    
    # Agrupar por ARTICULO_ID, LOCATION_ID, ANIO y MES para calcular las ventas totales
    # Asumimos que EXPORTACION representa unidades vendidas
    df_ventas_agrupado = df_ventas.groupby(['ARTICULO_ID', 'LOCATION_ID', 'ANIO', 'MES'])[
        'EXPORTACION'].sum().reset_index()
    
    # Renombrar columnas para consistencia con el ejemplo
    df_ventas_agrupado = df_ventas_agrupado.rename(columns={
        'ARTICULO_ID': 'articulo_id',
        'LOCATION_ID': 'location_id',
        'ANIO': 'year',
        'MES': 'month',
        'EXPORTACION': 'unidades_vendidas'
    })
    
    # Crear columna de fecha para facilitar el análisis temporal
    df_ventas_agrupado['date'] = pd.to_datetime(
        df_ventas_agrupado['year'].astype(str) + '-' + 
        df_ventas_agrupado['month'].astype(str).str.zfill(2) + '-01'
    )
    
    print("=== Ventas agrupadas (primeras filas) ===")
    display(df_ventas_agrupado.head())
    print(f"Total de registros: {len(df_ventas_agrupado)}")

=== Ventas agrupadas (primeras filas) ===


,articulo_id,location_id,year,month,unidades_vendidas,date
0,1,8,2025,4,160,2025-04-01
1,2,8,2025,4,130,2025-04-01


Total de registros: 2


In [19]:
# Feature Engineering

# Podemos proceder con el feature engineering si tenemos datos
if 'df_ventas_agrupado' in globals() and len(df_ventas_agrupado) > 0:
    # Crear copia para el modelado
    df_model = df_ventas_agrupado.copy()
    
    # Transformaciones cíclicas para el mes (capturar estacionalidad)
    df_model['month_sin'] = np.sin(2 * np.pi * (df_model['month'] - 1) / 12)
    df_model['month_cos'] = np.cos(2 * np.pi * (df_model['month'] - 1) / 12)
    
    # Aplicar encoding a variables categóricas
    le_loc = LabelEncoder()
    le_prod = LabelEncoder()
    df_model['loc_enc'] = le_loc.fit_transform(df_model['location_id'])
    df_model['prod_enc'] = le_prod.fit_transform(df_model['articulo_id'])
    
    # Mostrar resultado del feature engineering
    print("=== Dataset con feature engineering ===")
    display(df_model.head())
    
    # Definir características (features) y variable objetivo
    features = ['loc_enc', 'prod_enc', 'year', 'month', 'month_sin', 'month_cos']
    target = 'unidades_vendidas'
    
    print(f"Features seleccionados: {features}")
    print(f"Variable objetivo: {target}")
else:
    print("No hay datos suficientes para realizar feature engineering")

=== Dataset con feature engineering ===


,articulo_id,location_id,year,month,unidades_vendidas,date,month_sin,month_cos,loc_enc,prod_enc
0,1,8,2025,4,160,2025-04-01,1.0,6.123234e-17,0,0
1,2,8,2025,4,130,2025-04-01,1.0,6.123234e-17,0,1


Features seleccionados: ['loc_enc', 'prod_enc', 'year', 'month', 'month_sin', 'month_cos']
Variable objetivo: unidades_vendidas


In [20]:
# Entrenamiento del modelo y predicciones

if 'df_model' in globals() and 'features' in globals() and len(df_model) > 0:
    # Identificar el último período de datos para separar entrenamiento y prueba
    ultimo_periodo = df_model['date'].max()
    print(f"Último período disponible: {ultimo_periodo}")
    
    # Separar datos de entrenamiento (todos menos el último mes)
    train_data = df_model[df_model['date'] < ultimo_periodo]
    test_data = df_model[df_model['date'] == ultimo_periodo].copy()
    
    print(f"Datos de entrenamiento: {len(train_data)} registros")
    print(f"Datos de prueba: {len(test_data)} registros")
    
    # Verificar que hay suficientes datos para entrenamiento
    if len(train_data) > 10:  # Umbral arbitrario, ajustar según necesidad
        # Preparar conjuntos de entrenamiento
        X_train = train_data[features]
        y_train = train_data[target]
        
        # Entrenar modelo XGBoost
        model = XGBRegressor(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.1,
            random_state=42
        )
        model.fit(X_train, y_train)
        print("Modelo XGBoost entrenado correctamente")
        
        # Evaluar en datos de prueba si hay disponibles
        if len(test_data) > 0:
            X_test = test_data[features]
            y_test = test_data[target]
            y_pred = model.predict(X_test)
            
            # Convertir predicciones a enteros
            test_data['unidades_pred'] = y_pred.round().astype(int)
            
            # Mostrar métricas de precisión
            from sklearn.metrics import mean_absolute_error, mean_squared_error
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            
            print(f"Resultados de evaluación:")
            print(f"Error Absoluto Medio (MAE): {mae:.2f}")
            print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f}")
            
            # Mostrar predicciones vs valores reales
            comparison = test_data[['articulo_id', 'location_id', 'date', 'unidades_vendidas', 'unidades_pred']]
            display(comparison.head(10))
        else:
            print("No hay datos de prueba disponibles")
        
        # Preparar próximo periodo para predicción
        next_month_date = ultimo_periodo + pd.DateOffset(months=1)
        next_year = next_month_date.year
        next_month = next_month_date.month
        
        # Obtener lista única de articulos y locations
        unique_articles = df_model['articulo_id'].unique()
        unique_locations = df_model['location_id'].unique()
        
        print(f"\nPreparando predicciones para el próximo período: {next_month_date.strftime('%Y-%m-%d')}")
        print(f"Total de artículos: {len(unique_articles)}")
        print(f"Total de ubicaciones: {len(unique_locations)}")
        
        # Crear datos para predicción futura
        pred_list = []
        for loc in unique_locations:
            for prod in unique_articles:
                pred_list.append({
                    'location_id': loc,
                    'articulo_id': prod,
                    'date': next_month_date,
                    'year': next_year,
                    'month': next_month,
                    'month_sin': np.sin(2 * np.pi * (next_month - 1) / 12),
                    'month_cos': np.cos(2 * np.pi * (next_month - 1) / 12),
                    'loc_enc': le_loc.transform([loc])[0],
                    'prod_enc': le_prod.transform([prod])[0]
                })
        
        # Crear DataFrame para predicción
        df_next = pd.DataFrame(pred_list)
        X_next = df_next[features]
        
        # Hacer predicciones para el próximo período
        df_next['unidades_pred'] = model.predict(X_next).round().astype(int)
        
        print("\n=== Predicciones para el próximo período ===")
        display(df_next[['articulo_id', 'location_id', 'date', 'unidades_pred']].head(10))
        print(f"Total de predicciones: {len(df_next)}")
    else:
        print("No hay suficientes datos para entrenamiento")
else:
    print("No hay datos de modelo disponibles para entrenamiento")

Último período disponible: 2025-04-01 00:00:00
Datos de entrenamiento: 0 registros
Datos de prueba: 2 registros
No hay suficientes datos para entrenamiento


In [ ]:
# Análisis de inventario y generación de alertas

if 'df_next' in globals() and len(df_next) > 0:
    # Obtener datos de inventario actual
    # Nota: En este caso, ya tenemos df con esta información
    
    # Extraer las columnas relevantes de inventario
    df_inventory = df[['Articulo_ID', 'Location_ID', 'StockActual', 'StockMinimo']].copy()
    
    # Renombrar columnas para consistencia
    df_inventory = df_inventory.rename(columns={
        'Articulo_ID': 'articulo_id',
        'Location_ID': 'location_id',
        'StockActual': 'stock_actual',
        'StockMinimo': 'stock_min_actual'
    })
    
    # Eliminar duplicados si hay alguno (manteniendo un registro por artículo/ubicación)
    df_inventory = df_inventory.drop_duplicates(subset=['articulo_id', 'location_id'])
    
    print("=== Datos de inventario actual ===")
    display(df_inventory.head())
    
    # Unir datos de inventario con predicciones
    df_update = df_next.merge(df_inventory, on=['articulo_id', 'location_id'], how='left')
    
    # Calcular nuevo stock mínimo recomendado basado en predicciones
    # Añadimos un margen de seguridad del 20%
    df_update['stock_min_nuevo'] = (df_update['unidades_pred'] * 1.2).round().astype(int)
    
    # Generar alertas cuando el stock actual es menor que el stock mínimo nuevo
    df_update['alerta'] = df_update['stock_actual'] < df_update['stock_min_nuevo']
    
    # Mostrar resultados finales
    print("\n=== Análisis de inventario con alertas ===")
    display(df_update[['articulo_id', 'location_id', 'date', 
                      'stock_actual', 'stock_min_actual', 'unidades_pred',
                      'stock_min_nuevo', 'alerta']].head(10))
    
    # Mostrar solo productos con alerta
    productos_alerta = df_update[df_update['alerta'] == True]
    print(f"\n=== Productos que requieren atención ({len(productos_alerta)} productos) ===")
    if len(productos_alerta) > 0:
        display(productos_alerta[['articulo_id', 'location_id', 
                                 'stock_actual', 'stock_min_nuevo', 'unidades_pred']])
    else:
        print("No hay productos que requieran atención en este momento")
    
    # Calcular métricas generales
    total_articulos = len(df_update)
    porcentaje_alertas = (len(productos_alerta) / total_articulos * 100) if total_articulos > 0 else 0
    
    print(f"\nResumen:")
    print(f"- Total de productos analizados: {total_articulos}")
    print(f"- Productos que requieren reabastecimiento: {len(productos_alerta)} ({porcentaje_alertas:.1f}%)")
else:
    print("No hay datos de predicción disponibles para análisis de inventario")

In [ ]:
# Visualización de resultados y análisis gráfico

if 'df_update' in globals() and len(df_update) > 0:
    # 1. Gráfico de barras: Stock actual vs. Stock mínimo nuevo por ubicación
    plt.figure(figsize=(14, 8))
    
    # Agrupar por ubicación
    location_summary = df_update.groupby('location_id').agg({
        'stock_actual': 'sum',
        'stock_min_nuevo': 'sum',
        'alerta': 'sum'
    }).reset_index()
    
    location_summary['porcentaje_alertas'] = location_summary['alerta'] / df_update.groupby('location_id').size() * 100
    
    # Gráfico de barras agrupadas
    x = np.arange(len(location_summary))
    width = 0.35
    
    fig, ax1 = plt.subplots(figsize=(12, 6))
    
    # Barras para stock actual y stock mínimo
    bars1 = ax1.bar(x - width/2, location_summary['stock_actual'], width, label='Stock Actual')
    bars2 = ax1.bar(x + width/2, location_summary['stock_min_nuevo'], width, label='Stock Mínimo Nuevo')
    
    # Configurar eje Y1
    ax1.set_ylabel('Unidades')
    ax1.set_title('Análisis de Stock por Ubicación')
    ax1.set_xticks(x)
    ax1.set_xticklabels(location_summary['location_id'])
    ax1.legend(loc='upper left')
    
    # Añadir valores a las barras
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax1.annotate(f'{height}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom')
    
    add_labels(bars1)
    add_labels(bars2)
    
    # Segundo eje Y para porcentaje de alertas
    ax2 = ax1.twinx()
    ax2.plot(x, location_summary['porcentaje_alertas'], 'ro-', label='% Productos en Alerta')
    ax2.set_ylabel('Porcentaje de Productos en Alerta (%)')
    ax2.legend(loc='upper right')
    
    fig.tight_layout()
    plt.show()
    
    # 2. Histograma de la diferencia entre stock actual y predicción
    plt.figure(figsize=(12, 6))
    df_update['diferencia_stock'] = df_update['stock_actual'] - df_update['unidades_pred']
    
    # Determinar un umbral para colorear
    df_update['estado'] = 'Adecuado'
    df_update.loc[df_update['diferencia_stock'] < 0, 'estado'] = 'Déficit'
    df_update.loc[df_update['diferencia_stock'] > df_update['unidades_pred'], 'estado'] = 'Exceso'
    
    # Crear paleta de colores
    colors = {'Déficit': 'red', 'Adecuado': 'green', 'Exceso': 'orange'}
    
    # Agrupar por estado para histograma
    for estado, grupo in df_update.groupby('estado'):
        plt.hist(grupo['diferencia_stock'], alpha=0.6, label=estado, color=colors[estado], bins=15)
    
    plt.axvline(0, color='k', linestyle='--')
    plt.title('Distribución de la Diferencia entre Stock Actual y Ventas Predichas')
    plt.xlabel('Stock Actual - Ventas Predichas')
    plt.ylabel('Número de Productos')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # 3. Gráfico de dispersión: Stock actual vs Predicción, coloreado por alerta
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(df_update['unidades_pred'], 
                         df_update['stock_actual'],
                         c=df_update['alerta'].astype(int),
                         cmap='coolwarm', alpha=0.6, s=50)
    
    # Añadir línea diagonal de referencia (y=x)
    max_val = max(df_update['unidades_pred'].max(), df_update['stock_actual'].max()) * 1.1
    plt.plot([0, max_val], [0, max_val], 'k--', alpha=0.3)
    
    plt.colorbar(scatter, label='Alerta (1=Sí, 0=No)')
    plt.title('Stock Actual vs. Ventas Predichas')
    plt.xlabel('Ventas Predichas para el Próximo Período')
    plt.ylabel('Stock Actual')
    plt.grid(True, alpha=0.3)
    
    # Añadir etiquetas para algunos puntos destacados (top 5 alertas)
    highlight = df_update[df_update['alerta']].sort_values('stock_min_nuevo', ascending=False).head(5)
    for i, row in highlight.iterrows():
        plt.annotate(f"{row['articulo_id']}/{row['location_id']}",
                    (row['unidades_pred'], row['stock_actual']),
                    xytext=(5, 5), textcoords='offset points')
    
    plt.tight_layout()
    plt.show()
else:
    print("No hay datos suficientes para generar visualizaciones")

In [ ]:
# Recomendaciones y conclusiones automatizadas

if 'df_update' in globals() and len(df_update) > 0:
    # Generar insights automáticos basados en los resultados
    
    # 1. Productos críticos (mayor diferencia negativa entre stock actual y predicción)
    productos_criticos = df_update[df_update['alerta']].sort_values(
        by=['stock_actual'] - df_update[df_update['alerta']]['unidades_pred']
    ).head(5)
    
    # 2. Productos con exceso de stock
    productos_exceso = df_update[df_update['stock_actual'] > 2 * df_update['unidades_pred']].sort_values(
        by='stock_actual', ascending=False
    ).head(5)
    
    # 3. Cálculo de métricas generales de inventario
    valor_total_inventario = (df_update['stock_actual'] * df_update['unidades_pred']).sum()
    valor_productos_alerta = df_update[df_update['alerta']]['unidades_pred'].sum()
    productos_sin_rotacion = len(df_update[df_update['unidades_pred'] == 0])
    
    # 4. Análisis por ubicación
    ubicaciones_criticas = df_update.groupby('location_id')['alerta'].mean().sort_values(ascending=False)
    
    # Mostrar insights
    print("\n=== RECOMENDACIONES Y CONCLUSIONES AUTOMATIZADAS ===")
    
    print("\n1. PRODUCTOS QUE REQUIEREN ATENCIÓN INMEDIATA:")
    if len(productos_criticos) > 0:
        display(productos_criticos[['articulo_id', 'location_id', 'stock_actual', 
                                   'unidades_pred', 'stock_min_nuevo']])
        
        # Calcular cuánto inventario adicional se necesita
        productos_criticos['inventario_adicional'] = productos_criticos['stock_min_nuevo'] - productos_criticos['stock_actual']
        inventario_adicional_total = productos_criticos['inventario_adicional'].sum()
        
        print(f"\nSe necesita adquirir un total de {inventario_adicional_total} unidades adicionales")
        print("para los productos críticos identificados.")
    else:
        print("No se identificaron productos que requieran atención inmediata.")
    
    print("\n2. PRODUCTOS CON EXCESO DE INVENTARIO:")
    if len(productos_exceso) > 0:
        display(productos_exceso[['articulo_id', 'location_id', 'stock_actual', 
                                'unidades_pred']])
        
        # Calcular exceso
        productos_exceso['exceso_inventario'] = productos_exceso['stock_actual'] - productos_exceso['stock_min_nuevo']
        exceso_total = productos_exceso['exceso_inventario'].sum()
        
        print(f"\nSe identificó un exceso de {exceso_total} unidades en total para los productos")
        print("con mayor sobrestock. Considere redistribuir este inventario o aplicar estrategias")
        print("de venta para reducir el exceso.")
    else:
        print("No se identificaron productos con exceso significativo de inventario.")
    
    print("\n3. ANÁLISIS POR UBICACIÓN:")
    display(pd.DataFrame(ubicaciones_criticas).rename(columns={'alerta': 'Porcentaje_Alertas'}))
    ubicacion_mas_critica = ubicaciones_criticas.idxmax()
    print(f"\nLa ubicación '{ubicacion_mas_critica}' es la más crítica con {ubicaciones_criticas.max()*100:.1f}% de")
    print("productos en alerta. Se recomienda priorizar las acciones de inventario en esta ubicación.")
    
    # 5. Mostrar recomendaciones generales
    print("\n4. RECOMENDACIONES GENERALES:")
    print("- Revisar los niveles de stock mínimos actuales y ajustarlos basados en las predicciones.")
    print("- Priorizar el reabastecimiento de los productos críticos identificados.")
    print("- Considerar promociones para productos con exceso de inventario.")
    print("- Analizar la posibilidad de redistribuir inventario entre ubicaciones.")
    print("- Establecer un sistema de alertas automáticas basado en las predicciones mensuales.")
    
    # 6. Impacto financiero estimado
    print("\n5. IMPACTO FINANCIERO ESTIMADO:")
    print(f"- Valor total del inventario analizado: {valor_total_inventario} unidades")
    print(f"- Valor de productos en riesgo de stockout: {valor_productos_alerta} unidades ({valor_productos_alerta/valor_total_inventario*100:.1f}%)")
    print(f"- Productos sin rotación prevista: {productos_sin_rotacion} ({productos_sin_rotacion/len(df_update)*100:.1f}% del catálogo)")
else:
    print("No hay datos suficientes para generar recomendaciones")

In [ ]:
# Guardar el modelo y resultados para su uso posterior

import pickle
import os

if 'model' in globals() and 'df_update' in globals():
    # Crear directorio de modelos si no existe
    models_dir = '../models'
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)
    
    # Guardar el modelo entrenado
    model_path = os.path.join(models_dir, 'inventory_prediction_model.pkl')
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    
    # Guardar encoders para uso futuro
    encoders_path = os.path.join(models_dir, 'inventory_encoders.pkl')
    encoders = {
        'location_encoder': le_loc,
        'product_encoder': le_prod
    }
    with open(encoders_path, 'wb') as f:
        pickle.dump(encoders, f)
    
    # Guardar resultados como CSV
    results_dir = '../models/results'
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    
    # Guardar predicciones
    predictions_path = os.path.join(results_dir, f'predictions_{datetime.datetime.now().strftime("%Y%m%d")}.csv')
    df_update.to_csv(predictions_path, index=False)
    
    # Guardar alertas
    alertas_path = os.path.join(results_dir, f'alertas_{datetime.datetime.now().strftime("%Y%m%d")}.csv')
    if 'productos_alerta' in globals() and len(productos_alerta) > 0:
        productos_alerta.to_csv(alertas_path, index=False)
    
    print(f"Modelo guardado en: {model_path}")
    print(f"Encoders guardados en: {encoders_path}")
    print(f"Predicciones guardadas en: {predictions_path}")
    print(f"Alertas guardadas en: {alertas_path if 'productos_alerta' in globals() and len(productos_alerta) > 0 else 'No hay alertas para guardar'}")
else:
    print("No hay modelo o resultados para guardar")

In [ ]:
# Función para manejo de errores en la consulta

def verificar_resultados_consulta(df_result):
    """
    Verifica y reporta problemas en los resultados de la consulta
    """
    # Verificar que hay datos
    if df_result is None or len(df_result) == 0:
        print("⚠️ ADVERTENCIA: No se recuperaron datos de la consulta")
        return False
    
    # Verificar columnas críticas
    columnas_criticas = ['Inventario_ID', 'Articulo_ID', 'StockActual', 'StockMinimo', 
                        'Anio', 'Mes', 'Importacion', 'Exportacion']
    
    columnas_faltantes = [col for col in columnas_criticas if col not in df_result.columns]
    
    if columnas_faltantes:
        print(f"⚠️ ADVERTENCIA: Faltan columnas críticas en el resultado: {columnas_faltantes}")
        return False
    
    # Verificar si hay datos suficientes para el análisis
    min_registros = 10  # Número mínimo de registros para un análisis significativo
    if len(df_result) < min_registros:
        print(f"⚠️ ADVERTENCIA: Datos insuficientes para análisis. Se encontraron {len(df_result)} registros, pero se necesitan al menos {min_registros}.")
        return False
    
    # Verificar valores nulos en columnas críticas
    for col in columnas_criticas:
        if col in df_result.columns and df_result[col].isnull().sum() > 0:
            pct_nulos = df_result[col].isnull().mean() * 100
            print(f"⚠️ ADVERTENCIA: La columna '{col}' tiene {df_result[col].isnull().sum()} valores nulos ({pct_nulos:.1f}%).")
    
    print("✅ Verificación de resultados completada")
    return True

# Aplicar verificación a los datos ya cargados
if 'df' in globals() and df is not None:
    print("\nVerificando calidad de los datos cargados...")
    verificar_resultados_consulta(df)
else:
    print("\n⚠️ No hay datos cargados para verificar")

# Para usar en celdas posteriores
def ejecutar_si_hay_datos(df, mensaje_error="No hay datos suficientes para continuar"):
    """
    Decorador para ejecutar una función solo si hay datos suficientes
    """
    def decorador(func):
        def wrapper(*args, **kwargs):
            if df is None or len(df) == 0:
                print(f"⚠️ {mensaje_error}")
                return None
            return func(*args, **kwargs)
        return wrapper
    return decorador

In [ ]:
# Check the column names in each DataFrame to identify the correct case/format
print("Columns in inventario_df:", inventario_df.columns.tolist())
print("Columns in historial_df:", historial_df.columns.tolist())
print("Columns in articulo_df:", articulo_df.columns.tolist())
print("Columns in ordenes_productos_df:", ordenes_productos_df.columns.tolist())
print("Columns in ordenes_df:", ordenes_df.columns.tolist())

# Weekly Automated Stock Minimum Update Pipeline

This notebook has been extended with an automated weekly pipeline that:

1. **Retrains the model** every week using the most up-to-date data from the database
2. **Predicts demand** for the upcoming week for each product-location combination
3. **Updates the `stock_minimo` values** in the database with calculated values (with safety margin)

## Implementation Files

The automated pipeline has been implemented in the following files:

- `mlops/pipelines/weekly_stock_update.py`: The main pipeline script that connects to the database, trains the model, makes predictions, and updates the database
- `mlops/utils/schedule_stock_updates.py`: A scheduler script that runs the pipeline weekly (configured for Monday at 1:00 AM)

## How to Run

To manually run the pipeline once:
```
python mlops/pipelines/weekly_stock_update.py
```

To start the weekly scheduler:
```
python mlops/utils/schedule_stock_updates.py
```

## Output Files

The pipeline generates the following output files:

- Trained models are saved in `mlops/models/` as pickle files
- Prediction results are saved in `mlops/models/results/` as CSV files
- Logs are stored in `mlops/logs/` for monitoring

See the README.md file in the mlops directory for more details.

# Integración con la Aplicación Web

El modelo de predicción de stock mínimo ha sido integrado con la aplicación web para permitir su ejecución programada y manual desde la interfaz de usuario:

## Componentes de la Integración

### Backend

1. **Programador de Tareas**
   - Implementado en `backend/services/stockUpdateScheduler.js`
   - Utiliza node-cron para programar la ejecución semanal del modelo
   - Lee la configuración desde el archivo Python de configuración

2. **API Endpoints**
   - Rutas definidas en `backend/routes/ml.js`
   - Controladores en `backend/controllers/mlModelController.js`
   - Funcionalidades:
     - Ejecución manual del pipeline
     - Visualización de logs
     - Consulta de próxima ejecución programada

3. **Inicialización en Servidor**
   - El programador se inicia automáticamente al arrancar el servidor
   - Configurado en `backend/server.js`

### Frontend

Se ha desarrollado una interfaz de usuario para administradores en:
`frontend/src/pages/admin/ml/ModelManagement.jsx`

Esta interfaz permite:
- Ver cuándo será la próxima actualización automática
- Ejecutar manualmente la actualización del modelo
- Visualizar los logs de ejecución

## Diagrama de Flujo

```
[Aplicación Web] ---- Programación ---> [Node Cron] ---- Ejecución ---> [Python Script] ---- Actualización ---> [Base de Datos]
       |
       +----- API Manual -----> [Python Script] ---- Actualización ---> [Base de Datos]
```

## Beneficios de la Integración

1. **Automatización completa**: El modelo se ejecuta sin intervención humana
2. **Supervisión centralizada**: Los administradores pueden monitorear desde la aplicación
3. **Ejecución manual**: Permite actualizaciones bajo demanda cuando sea necesario
4. **Transparencia**: Los logs son accesibles desde la interfaz web

# Generación de Datos Sintéticos para el Modelo

En esta sección crearemos datos sintéticos que imitan los datos reales para entrenar y probar el modelo de predicción de demanda e inventario. Utilizaremos la biblioteca Faker para generar datos realistas.

In [ ]:
# Instalar la biblioteca Faker si no está disponible
!pip install faker

In [ ]:
# Script de generación de datos sintéticos para el modelo de predicción

import pandas as pd
import numpy as np
from faker import Faker
import datetime
import random
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración inicial
faker = Faker(['es_MX'])  # Configurar para español de México
random.seed(42)  # Para reproducibilidad
np.random.seed(42)
fake = Faker()

# Parámetros de generación
num_locations = 5        # Número de ubicaciones/tiendas
num_articles = 30        # Número de artículos/productos
num_users = 15           # Número de usuarios
months_of_data = 24      # Meses de historial a generar
start_date = datetime.date(2023, 6, 1)  # Fecha inicial

# Función para generar datos de ubicación
def generate_locations(num_locations):
    locations = []
    location_types = ['Tienda', 'Almacén', 'Centro Distribución', 'Fábrica']
    
    for i in range(1, num_locations + 1):
        location = {
            'Location_ID': i,
            'Nombre': faker.company(),
            'Tipo': random.choice(location_types),
            'PosicionX': random.randint(1, 1000),
            'PosicionY': random.randint(1, 1000),
            'FechaCreado': faker.date_between(start_date='-3y', end_date='today')
        }
        locations.append(location)
    
    return pd.DataFrame(locations)

# Función para generar datos de artículos
def generate_articles(num_articles):
    articles = []
    categories = ['Electrónica', 'Ropa', 'Hogar', 'Alimentos', 'Herramientas', 'Juguetes']
    seasons = ['Verano', 'Invierno', 'Primavera', 'Otoño', 'Todo el año']
    
    for i in range(1, num_articles + 1):
        category = random.choice(categories)
        
        # Precios realistas según la categoría
        if category == 'Electrónica':
            supplier_price = round(random.uniform(500, 5000), 2)
        elif category == 'Ropa':
            supplier_price = round(random.uniform(100, 800), 2)
        elif category == 'Alimentos':
            supplier_price = round(random.uniform(10, 200), 2)
        else:
            supplier_price = round(random.uniform(50, 1500), 2)
        
        # Margen de ganancia entre 20% y 60%
        margin = random.uniform(1.2, 1.6)
        sale_price = round(supplier_price * margin, 2)
        
        article = {
            'Articulo_ID': i,
            'Nombre': f"{faker.word().capitalize()} {random.choice(['Pro', 'Max', 'Plus', 'Lite', ''])}",
            'Categoria': category,
            'PrecioProveedor': supplier_price,
            'PrecioVenta': sale_price,
            'Temporada': random.choice(seasons)
        }
        articles.append(article)
    
    return pd.DataFrame(articles)

# Función para generar datos de inventario base
def generate_inventory(locations_df, articles_df):
    inventory = []
    inventory_id = 1
    
    for loc_id in locations_df['Location_ID']:
        for art_id in articles_df['Articulo_ID']:
            # Obtener categoría del artículo para ajustar los valores
            category = articles_df[articles_df['Articulo_ID'] == art_id]['Categoria'].values[0]
            
            # Ajustar stock según categoría
            if category == 'Electrónica':
                stock_actual = random.randint(5, 50)
                stock_min = random.randint(3, 10)
            elif category == 'Alimentos':
                stock_actual = random.randint(20, 200)
                stock_min = random.randint(10, 30)
            else:
                stock_actual = random.randint(10, 100)
                stock_min = random.randint(5, 20)
                
            # Calcular stock recomendado (1.5-2x el mínimo)
            stock_recom = int(stock_min * random.uniform(1.5, 2.0))
            
            # Generar valores de importación/exportación realistas
            importacion = random.randint(0, 100)
            exportacion = random.randint(0, stock_actual // 2)  # No más de la mitad del stock
            
            # Generar fechas realistas
            fecha_importacion = faker.date_between(start_date='-6m', end_date='today')
            fecha_exportacion = faker.date_between(start_date=fecha_importacion, end_date='today')
            
            # Generar margen de ganancia realista (20%-60%)
            margin = round(random.uniform(0.20, 0.60), 2)
            
            # Generar tiempo reposición (1-14 días)
            tiempo_repos = round(random.uniform(1, 14), 1)
            
            # Stock de seguridad (20%-40% del stock mínimo)
            stock_seguridad = int(stock_min * random.uniform(0.2, 0.4))
            
            # Demanda promedio (varía según categoría)
            if category == 'Electrónica':
                demanda = round(random.uniform(1, 10), 1)
            elif category == 'Alimentos':
                demanda = round(random.uniform(5, 30), 1)
            else:
                demanda = round(random.uniform(2, 15), 1)
            
            inventory_item = {
                'Inventario_ID': inventory_id,
                'Articulo_ID': art_id,
                'Location_ID': loc_id,
                'StockActual': stock_actual,
                'Importacion': importacion,
                'Exportacion': exportacion,
                'StockMinimo': stock_min,
                'StockRecomendado': stock_recom,
                'FechaUltimaImportacion': fecha_importacion,
                'FechaUltimaExportacion': fecha_exportacion,
                'MargenGanancia': margin,
                'TiempoReposicion': tiempo_repos,
                'StockSeguridad': stock_seguridad,
                'DemandaPromedio': demanda
            }
            
            inventory.append(inventory_item)
            inventory_id += 1
    
    return pd.DataFrame(inventory)

# Función para generar historial de productos con tendencias y estacionalidad
def generate_product_history(inventory_df, months=24):
    history = []
    current_date = start_date
    
    # Crear un historial id inicial
    history_id = 1
    
    # Para cada mes en el rango
    for month_offset in range(months):
        # Calcular fecha actual
        current_date = start_date + timedelta(days=30*month_offset)
        year = current_date.year
        month = current_date.month
        
        # Factor de estacionalidad mensual (mayor demanda en ciertos meses)
        seasonality = {
            1: 1.2,  # Enero (post-navidad, liquidaciones)
            2: 0.8,  # Febrero
            3: 0.9,  # Marzo
            4: 1.0,  # Abril
            5: 1.1,  # Mayo
            6: 1.2,  # Junio (verano)
            7: 1.3,  # Julio (verano)
            8: 1.3,  # Agosto (regreso a clases)
            9: 1.1,  # Septiembre
            10: 1.0,  # Octubre
            11: 1.2,  # Noviembre (Black Friday)
            12: 1.5   # Diciembre (navidad)
        }
        
        # Para cada registro en el inventario
        for _, inv in inventory_df.iterrows():
            # Obtener artículo y datos relacionados
            article_id = inv['Articulo_ID']
            location_id = inv['Location_ID']
            stock_inicial = inv['StockActual']
            
            # Aplicar tendencia y estacionalidad para importaciones/exportaciones
            # Tendencia: aumento gradual a lo largo del tiempo (5% por año)
            trend_factor = 1 + (0.05 * (month_offset / 12))
            
            # Aplicar factores a las importaciones
            base_importacion = random.randint(5, 50)
            importacion = int(base_importacion * trend_factor * seasonality[month])
            
            # Aplicar factores a las exportaciones/ventas
            base_exportacion = random.randint(3, max(4, int(importacion * 0.8)))
            exportacion = int(base_exportacion * trend_factor * seasonality[month])
            
            # A veces hay picos aleatorios de demanda
            if random.random() < 0.1:  # 10% de probabilidad de un pico
                exportacion = int(exportacion * random.uniform(1.5, 3.0))
            
            # Calcular stock final
            if month_offset == 0:
                # Para el primer mes, usar el stock actual como inicial
                stock_start = stock_inicial
            else:
                # Buscar el stock final del mes anterior
                prev_records = [h for h in history if h['Inventario_ID'] == inv['Inventario_ID'] 
                              and h['Year'] == (year if month > 1 else year-1) 
                              and h['Month'] == (month-1 if month > 1 else 12)]
                
                if prev_records:
                    stock_start = prev_records[0]['StockEnd']
                else:
                    stock_start = stock_inicial
            
            # Calcular stock final
            stock_end = max(0, stock_start + importacion - exportacion)
            
            history_record = {
                'Historial_ID': history_id,
                'Inventario_ID': inv['Inventario_ID'],
                'Location_ID': location_id,
                'Year': year,
                'Month': month,
                'Importacion': importacion,
                'Exportacion': exportacion,
                'StockStart': stock_start,
                'StockEnd': stock_end
            }
            
            history.append(history_record)
            history_id += 1
    
    return pd.DataFrame(history)

# Función para generar ordenes
def generate_orders(inventory_df, users_df, months=24):
    orders = []
    order_products = []
    order_id = 1
    order_product_id = 1
    
    order_types = ['Compra', 'Venta', 'Transferencia', 'Devolución']
    order_states = ['Creada', 'Aceptada', 'En Proceso', 'En Camino', 'Entregada', 'Cancelada']
    payment_methods = [1, 2, 3]  # IDs de métodos de pago
    
    # Obtener IDs de usuarios y roles
    user_ids = list(users_df['Usuario_ID'])
    user_roles = dict(zip(users_df['Usuario_ID'], users_df['Rol_ID']))
    
    # Generar órdenes para cada mes en el historial
    current_date = start_date
    
    for month_offset in range(months):
        # Calcular la cantidad de órdenes para el mes (entre 20 y 100)
        num_orders_in_month = random.randint(20, 100)
        
        # Fecha para el mes actual
        month_start_date = start_date + timedelta(days=30*month_offset)
        month_end_date = month_start_date + timedelta(days=29)
        
        for _ in range(num_orders_in_month):
            # Seleccionar fecha aleatoria dentro del mes
            order_date = faker.date_between(start_date=month_start_date, end_date=month_end_date)
            
            # Seleccionar usuarios para la orden
            creator_id = random.choice(user_ids)
            modifier_id = random.choice(user_ids)
            
            # Tipo de orden y estado
            order_type = random.choice(order_types)
            order_state = random.choice(order_states)
            
            # Fechas de proceso
            acceptance_date = None
            if order_state != 'Creada':
                acceptance_date = order_date + timedelta(days=random.randint(1, 3))
            
            payment_limit_date = order_date + timedelta(days=random.randint(7, 30))
            
            # Fecha estimada de entrega (más tiempo para compras que para ventas)
            if order_type == 'Compra':
                est_delivery_days = random.randint(5, 15)
            else:
                est_delivery_days = random.randint(1, 7)
            
            estimated_delivery_date = order_date + timedelta(days=est_delivery_days)
            
            # Fecha real de entrega
            actual_delivery_date = None
            on_time_delivery = None
            delivery_time = None
            
            if order_state == 'Entregada':
                # Variación aleatoria en la entrega (-2 a +5 días)
                delivery_variation = random.randint(-2, 5)
                actual_delivery_date = estimated_delivery_date + timedelta(days=delivery_variation)
                
                # ¿Entrega a tiempo?
                on_time_delivery = actual_delivery_date <= estimated_delivery_date
                
                # Tiempo de entrega en días
                delivery_time = (actual_delivery_date - order_date).days
            
            # Método de pago y descuento
            payment_method = random.choice(payment_methods)
            discount = round(random.uniform(0, 0.20), 2)  # 0-20% de descuento
            
            # Tiempo de reposición (1-14 días)
            restock_time = round(random.uniform(1, 14), 1)
            
            # Crear la orden
            order = {
                'Orden_ID': order_id,
                'Creado_por_ID': creator_id,
                'Modificado_por_ID': modifier_id,
                'TipoOrden': order_type,
                'Organizacion': faker.company(),
                'FechaCreacion': order_date,
                'FechaAceptacion': acceptance_date,
                'FechaLimitePago': payment_limit_date,
                'FechaEstimadaEntrega': estimated_delivery_date,
                'FechaEntrega': actual_delivery_date,
                'EntregaATiempo': on_time_delivery,
                'Estado': order_state,
                'MetodoPago_ID': payment_method,
                'DescuentoAplicado': discount,
                'TiempoReposicion': restock_time,
                'TiempoEntrega': delivery_time
            }
            
            orders.append(order)
            
            # Agregar productos a la orden (1-5 productos por orden)
            num_products = random.randint(1, 5)
            
            # Seleccionar productos aleatorios sin repetir
            inventory_sample = inventory_df.sample(n=min(num_products, len(inventory_df)))
            
            order_total = 0
            
            for _, inv in inventory_sample.iterrows():
                # Cantidad de productos
                quantity = random.randint(1, 5)
                
                # Buscar el precio del artículo
                article_id = inv['Articulo_ID']
                
                # Usar el precio de venta como precio unitario (podría ser diferente en un caso real)
                unit_price = inv['PrecioVenta'] if 'PrecioVenta' in inv else random.uniform(10, 1000)
                
                # Agregar producto a la orden
                order_product = {
                    'OrdenesProductos_ID': order_product_id,
                    'Orden_ID': order_id,
                    'Inventario_ID': inv['Inventario_ID'],
                    'Cantidad': quantity,
                    'PrecioUnitario': unit_price
                }
                
                order_products.append(order_product)
                order_product_id += 1
                
                # Sumar al total de la orden
                order_total += quantity * unit_price
            
            # Aplicar descuento al total
            order_total = round(order_total * (1 - discount), 2)
            
            # Actualizar total en la orden
            orders[-1]['Total'] = order_total
            
            order_id += 1
    
    return pd.DataFrame(orders), pd.DataFrame(order_products)

# Función para generar usuarios
def generate_users(num_users):
    users = []
    roles = [1, 2, 3, 4]  # IDs de roles
    
    for i in range(1, num_users + 1):
        role_id = random.choice(roles)
        location_id = random.randint(1, num_locations)
        
        user = {
            'Usuario_ID': i,
            'Correo': faker.email(),
            'Username': faker.user_name(),
            'Nombre': faker.name(),
            'Rol_ID': role_id,
            'Clave': faker.password(),
            'Location_ID': location_id,
            'FechaEmpiezo': faker.date_between(start_date='-2y', end_date='-1m'),
            'RFC': faker.bothify(text='???######???').upper()
        }
        users.append(user)
    
    return pd.DataFrame(users)

# Generar todos los datos
print("Generando datos...")

# 1. Generar ubicaciones
locations_df = generate_locations(num_locations)
print(f"- Ubicaciones generadas: {len(locations_df)}")

# 2. Generar artículos
articles_df = generate_articles(num_articles)
print(f"- Artículos generados: {len(articles_df)}")

# 3. Generar usuarios
users_df = generate_users(num_users)
print(f"- Usuarios generados: {len(users_df)}")

# 4. Generar inventario base
inventory_df = generate_inventory(locations_df, articles_df)
print(f"- Registros de inventario generados: {len(inventory_df)}")

# 5. Fusionar datos de artículos e inventario para órdenes de productos
inventory_enriched = inventory_df.merge(articles_df, on='Articulo_ID', how='left')

# 6. Generar historial de productos
history_df = generate_product_history(inventory_df, months=months_of_data)
print(f"- Registros de historial generados: {len(history_df)}")

# 7. Generar ordenes y productos en ordenes
orders_df, order_products_df = generate_orders(inventory_enriched, users_df, months=months_of_data)
print(f"- Ordenes generadas: {len(orders_df)}")
print(f"- Productos en ordenes generados: {len(order_products_df)}")

# Mostrar muestra de datos generados
def show_sample(df, title, n=5):
    print(f"\n{title} (muestra de {n} registros):")
    display(df.head(n))
    print(f"Total: {len(df)} registros")
    print(f"Columnas: {df.columns.tolist()}")

show_sample(locations_df, "Ubicaciones")
show_sample(articles_df, "Artículos")
show_sample(inventory_df, "Inventario")
show_sample(history_df, "Historial de Productos")
show_sample(orders_df, "Órdenes")
show_sample(order_products_df, "Productos en Órdenes")

# Visualización de algunos patrones en los datos

# 1. Gráfico de estacionalidad - Exportaciones por mes
plt.figure(figsize=(12, 6))
monthly_exports = history_df.groupby(['Year', 'Month'])['Exportacion'].sum().reset_index()
monthly_exports['Date'] = pd.to_datetime(monthly_exports['Year'].astype(str) + '-' + 
                                       monthly_exports['Month'].astype(str) + '-01')
monthly_exports = monthly_exports.sort_values('Date')

sns.lineplot(data=monthly_exports, x='Date', y='Exportacion')
plt.title('Patrón de Estacionalidad - Exportaciones Mensuales')
plt.xlabel('Fecha')
plt.ylabel('Unidades Exportadas')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 2. Distribución de Stock Actual vs Stock Mínimo
plt.figure(figsize=(10, 6))
sns.scatterplot(data=inventory_df, x='StockMinimo', y='StockActual', hue='Location_ID')
plt.title('Relación entre Stock Actual y Stock Mínimo')
plt.xlabel('Stock Mínimo')
plt.ylabel('Stock Actual')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 3. Gráfico de ordenes por tipo
plt.figure(figsize=(10, 6))
order_types = orders_df['TipoOrden'].value_counts()
sns.barplot(x=order_types.index, y=order_types.values)
plt.title('Distribución de Ordenes por Tipo')
plt.xlabel('Tipo de Orden')
plt.ylabel('Cantidad')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n¡Datos sintéticos generados exitosamente!")

# Opcional: Guardar los datos generados en archivos CSV
data_dir = '../models/synthetic_data'
import os
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

locations_df.to_csv(f"{data_dir}/locations.csv", index=False)
articles_df.to_csv(f"{data_dir}/articles.csv", index=False)
inventory_df.to_csv(f"{data_dir}/inventory.csv", index=False)
history_df.to_csv(f"{data_dir}/product_history.csv", index=False)
orders_df.to_csv(f"{data_dir}/orders.csv", index=False)
order_products_df.to_csv(f"{data_dir}/order_products.csv", index=False)
users_df.to_csv(f"{data_dir}/users.csv", index=False)

print(f"\nDatos guardados en: {data_dir}")

In [ ]:
# Generar sentencias SQL INSERT para cargar los datos en la base de datos HANA

def generate_insert_statements(df, table_name, schema='DBADMIN'):
    """Genera sentencias SQL INSERT a partir de un DataFrame"""
    all_inserts = []
    
    qualified_table = f'{schema}.{table_name}'
    
    # Generar una sentencia INSERT para cada fila
    for _, row in df.iterrows():
        columns = []
        values = []
        
        for col, val in row.items():
            if val is not None:  # Solo incluir valores no nulos
                columns.append(col)
                
                # Formatear el valor según su tipo
                if isinstance(val, str):
                    # Escapar comillas simples
                    val_str = f"'{val.replace("'", "''")}'" 
                elif isinstance(val, (datetime.date, datetime.datetime)):
                    val_str = f"'{val}'" 
                elif isinstance(val, bool):
                    val_str = '1' if val else '0'
                elif pd.isna(val):
                    val_str = 'NULL'
                else:
                    val_str = str(val)
                
                values.append(val_str)
        
        # Crear la sentencia INSERT
        if columns and values:
            columns_str = ", ".join(columns)
            values_str = ", ".join(values)
            insert_stmt = f"INSERT INTO {qualified_table} ({columns_str}) VALUES ({values_str});"
            all_inserts.append(insert_stmt)
    
    return all_inserts

# Generar sentencias INSERT para cada tabla
sql_inserts = {}
sql_inserts['Location2'] = generate_insert_statements(locations_df, 'Location2')
sql_inserts['Articulo2'] = generate_insert_statements(articles_df, 'Articulo2')
sql_inserts['Inventario2'] = generate_insert_statements(inventory_df, 'Inventario2')
sql_inserts['HistorialProductos2'] = generate_insert_statements(history_df, 'HistorialProductos2')
sql_inserts['Ordenes2'] = generate_insert_statements(orders_df, 'Ordenes2')
sql_inserts['OrdenesProductos2'] = generate_insert_statements(order_products_df, 'OrdenesProductos2')

# Combinar todas las sentencias en un solo script SQL
all_sql = []

for table, inserts in sql_inserts.items():
    all_sql.append(f"-- Insertar datos en {table}")
    all_sql.append(f"-- Total de registros: {len(inserts)}\n")
    
    # Añadir solo las primeras 10 sentencias como ejemplo
    for i, insert in enumerate(inserts[:10]):
        all_sql.append(insert)
    
    if len(inserts) > 10:
        all_sql.append(f"-- ... y {len(inserts) - 10} sentencias más ...")
    
    all_sql.append("\n")

# Guardar el script SQL completo en un archivo
sql_file_path = f"{data_dir}/insert_synthetic_data.sql"
with open(sql_file_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(all_sql))

print(f"Script SQL generado y guardado en: {sql_file_path}")

# Mostrar ejemplos de sentencias INSERT para cada tabla
print("\nEjemplos de sentencias INSERT:")

for table, inserts in sql_inserts.items():
    if inserts:
        print(f"\n{table} (ejemplo de 1 de {len(inserts)} sentencias):")
        print(inserts[0])

In [ ]:
# Exportar datos en formato JSONL para carga masiva en HANA

import json

def dataframe_to_jsonl(df, file_path):
    """Convierte un DataFrame a formato JSONL"""
    with open(file_path, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            # Convertir cada fila a un diccionario y luego a JSON
            row_dict = row.to_dict()
            
            # Convertir tipos de datos especiales a strings
            for key, value in row_dict.items():
                if isinstance(value, (datetime.date, datetime.datetime)):
                    row_dict[key] = value.isoformat()
                elif pd.isna(value):
                    row_dict[key] = None
            
            f.write(json.dumps(row_dict, ensure_ascii=False) + '\n')

# Exportar cada DataFrame a JSONL
jsonl_dir = f"{data_dir}/jsonl"
if not os.path.exists(jsonl_dir):
    os.makedirs(jsonl_dir)

dataframe_to_jsonl(locations_df, f"{jsonl_dir}/locations.jsonl")
dataframe_to_jsonl(articles_df, f"{jsonl_dir}/articles.jsonl")
dataframe_to_jsonl(inventory_df, f"{jsonl_dir}/inventory.jsonl")
dataframe_to_jsonl(history_df, f"{jsonl_dir}/product_history.jsonl")
dataframe_to_jsonl(orders_df, f"{jsonl_dir}/orders.jsonl")
dataframe_to_jsonl(order_products_df, f"{jsonl_dir}/order_products.jsonl")

print(f"\nDatos exportados en formato JSONL en: {jsonl_dir}")
print("Este formato es ideal para cargas masivas en HANA utilizando comandos IMPORT FROM.")

## Instrucciones para Cargar los Datos en HANA

Los datos sintéticos generados pueden cargarse en la base de datos HANA de varias maneras:

### 1. Carga mediante sentencias SQL INSERT

- Use el archivo `insert_synthetic_data.sql` generado
- Ejecute las sentencias SQL en HANA Studio o mediante la API de HANA
- Útil para conjuntos pequeños de datos o para pruebas específicas

### 2. Carga masiva utilizando archivos CSV

```sql
-- Ejemplo para cargar datos desde CSV
IMPORT FROM CSV FILE '/path/to/synthetic_data/inventory.csv' 
  INTO DBADMIN.INVENTARIO2
  WITH 
    RECORD DELIMITED BY '\n'
    FIELD DELIMITED BY ','
    OPTIONALLY ENCLOSED BY '"'
    SKIP FIRST ROW;
```

### 3. Carga masiva utilizando archivos JSONL

```sql
-- Ejemplo para cargar datos desde JSONL
IMPORT FROM JSONL FILE '/path/to/synthetic_data/jsonl/inventory.jsonl' 
  INTO DBADMIN.INVENTARIO2;
```

### Consideraciones importantes:

- Ajuste los nombres de las tablas y columnas según la configuración exacta de su base de datos
- Verifique las políticas de seguridad y permisos antes de cargar datos
- Para grandes volúmenes de datos, considere dividir la carga en lotes más pequeños
- Realice una copia de seguridad de los datos existentes antes de cargar nuevos datos

In [ ]:
# Mostrar las 10 variables más importantes para el modelo según correlación con StockRecomendado
# Esto puede servir para validar nuestro dataset antes de pasarlo al modelo

correlaciones = dataset_modelo.corr()['StockRecomendado'].sort_values(ascending=False)
plt.figure(figsize=(12, 8))
sns.barplot(x=correlaciones.values[1:11], y=correlaciones.index[1:11])
plt.title('Top 10 Variables más Correlacionadas con StockRecomendado')
plt.xlabel('Coeficiente de Correlación')
plt.ylabel('Variable')
plt.tight_layout()
plt.show()